This notebook utilizes ProteinBERT to create embeddings for protein sequences and classify them into seven distinct families. The embeddings are then used to train a classifier, significantly improving performance compared to previous methods.

In [ ]:
!pip install git+https://github.com/nadavbra/protein_bert.git

  Cloning https://github.com/nadavbra/protein_bert.git to /tmp/pip-req-build-l3aejpns
  Running command git clone --filter=blob:none --quiet https://github.com/nadavbra/protein_bert.git /tmp/pip-req-build-l3aejpns
  Resolved https://github.com/nadavbra/protein_bert.git to commit 168a4db5aac281ff14165d00e50f862d780a8966
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import pandas as pd
from IPython.display import display
from tensorflow import keras
from sklearn.model_selection import train_test_split
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len, log
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs


In [ ]:
BENCHMARK_NAME = 'scop'
OUTPUT_TYPE = OutputType(False, 'categorical')

In [ ]:
settings = {
    'max_dataset_size': None,
    'max_epochs_per_stage': 40,
    'seq_len': 512,
    'batch_size': 32,
    'final_epoch_seq_len': 1024,
    'initial_lr_with_frozen_pretrained_layers': 1e-02,
    'initial_lr_with_all_layers': 1e-04,
    'final_epoch_lr': 1e-05,
    'dropout_rate': 0.5,
    'training_callbacks': [
        keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
        keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
    ],
}

In [ ]:
def load_benchmark_dataset():
    train_set_url = 'https://raw.githubusercontent.com/nadavbra/protein_bert/master/protein_benchmarks/scop.train.csv'
    test_set_url = 'https://raw.githubusercontent.com/nadavbra/protein_bert/master/protein_benchmarks/scop.test.csv'

    train_set = pd.read_csv(train_set_url).dropna().drop_duplicates()
    test_set = pd.read_csv(test_set_url).dropna().drop_duplicates()

    train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=0)

    return train_set, valid_set, test_set


In [ ]:
train_set, valid_set, test_set=load_benchmark_dataset()

In [ ]:
train_set


,seq,label
7511,LPASIFRAYDIRGVVGDTLTAETAYWIGRAIGSESLARGEPCVAVG...,c
4727,LVPRGSHMNTSELRICRINKESGPCTGGEELYLLCDKVQKEDISVV...,b
6950,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,e
2250,GYFDAHALAMDYRSLGFRECLAEVARYLSIIEGLDASDPLRVRLVS...,a
8075,AKNVVLDHDGNLDDFVAMVLLASNTEKVRLIGALCTDADCFVENGF...,c
...,...,...
7600,PDVKCVCCTEGKECACFGQDCCVTGECCKDGTCCGI,g
8088,MKIISKEFTVKTRSRFDSIDITEQVSEAIKGINNGIAHVIVKHTTC...,d
6254,PHDPLDDIQADPWALWLSGRTRTALELPHYRRAAVLVALTREADPR...,d
15448,MFGNLQGKFIIATPEMDDEYFDRTVIYICEHNDNGTIGVIINTPTD...,d


In [ ]:
valid_set

,seq,label
11559,QPLEGYTLFSHRSAPNGFKVAIVLSELGFHYNTIFLDFNLGEHRAP...,c
11712,GICAPFTIPDVALEPGQQVTVPVAVTNQSGIAVPKPSLQLDASPDW...,b
10191,DDRRDALLERINLDIPAAVAQALREDLGGEVDAGNDITAQLLPADT...,d
1721,DIGQVIHPDDFDKAAADDYVLHEDGEKIYFLIKSKTDEYCFTNLAL...,b
5742,GPLSLSVDAFKILEDPKWEFPRKNLVLGKTLGEGEFGKVVKATAFH...,d
...,...,...
15461,EVVASNETLYQVVKEVKPGGLVQIADGTYKDVQLIVSNSGKSGLPI...,b
14052,QKSVLEQLKQVTMVVADTGDFELIKKYKPVDATTNPSLILKAVKEQ...,c
6884,SKMPQVNLRWPREVLDLVRKVAEENGRSVNSEIYQRVMESFKKEGRIGA,a
13128,PRLVALVKGRVQGVGYRAFAQKKALELGLSGYAENLPDGRVEVVAE...,d


In [ ]:
test_set

,seq,label
0,DPMTCEQAMASCEHTMCGYCQGPLYMTCIGITTDPECGLP,a
1,GSDKIHHHHHHMNIFEAIENRHSVRDFLERKMPERVKDDIENLLVK...,d
2,MPDLNSSTDSAASASAASDVSVESTAEATVCTVTLEKMSAGLGFSL...,b
3,MDFHIRKATNSDAEAIQHVATTSWHHTYQDLIPSDVQDDFLKRFYN...,d
4,DQSGYERGLTLPLRHPSGLFDGETEAVWGLNTAYSVVEKSVSTRDY...,b
...,...,...
3916,MKVEEILEKALELVIPDEEEVRKGREAEEELRRRLDELGVEYVFVG...,d
3917,MFEARLVQGSILKKVLEALKDLINEACWDISSSGVNLQSMDSSHVS...,d
3918,MEGEIDIAKRIEDGINQVQCSVAEYPEAITYLLEQYNRVEAEEARL...,a
3919,KESCKMFIGGLNWDTTEDNLREYFGKYGTVTDLKIMKDPATGRSRG...,d


Model

In [ ]:
def run_benchmark(benchmark_name, pretraining_model_generator, input_encoder, pretraining_model_manipulation_function = None):

    log('========== %s ==========' % benchmark_name)

    output_type = OutputType(False, 'categorical')
    log('Output type: %s' % output_type)

    train_set, valid_set, test_set = load_benchmark_dataset()
    log(f'{len(train_set)} training set records, {len(valid_set)} validation set records, {len(test_set)} test set records.')

    if settings['max_dataset_size'] is not None:
        log('Limiting the training, validation and test sets to %d records each.' % settings['max_dataset_size'])
        train_set = train_set.sample(min(settings['max_dataset_size'], len(train_set)), random_state = 0)
        valid_set = valid_set.sample(min(settings['max_dataset_size'], len(valid_set)), random_state = 0)
        test_set = test_set.sample(min(settings['max_dataset_size'], len(test_set)), random_state = 0)

    if output_type.is_seq or output_type.is_categorical:
        train_set['label'] = train_set['label'].astype(str)
        valid_set['label'] = valid_set['label'].astype(str)
        test_set['label'] = test_set['label'].astype(str)
    else:
        train_set['label'] = train_set['label'].astype(float)
        valid_set['label'] = valid_set['label'].astype(float)
        test_set['label'] = test_set['label'].astype(float)

    if output_type.is_categorical:

        if output_type.is_seq:
            unique_labels = sorted(set.union(*train_set['label'].apply(set)) | set.union(*valid_set['label'].apply(set)) | \
                    set.union(*test_set['label'].apply(set)))
        else:
            unique_labels = sorted(set(train_set['label'].unique()) | set(valid_set['label'].unique()) | set(test_set['label'].unique()))

        log('%d unique lebels.' % len(unique_labels))
    elif output_type.is_binary:
        unique_labels = [0, 1]
    else:
        unique_labels = None

    output_spec = OutputSpec(output_type, unique_labels)
    model_generator = FinetuningModelGenerator(pretraining_model_generator, output_spec, pretraining_model_manipulation_function = \
            pretraining_model_manipulation_function, dropout_rate = settings['dropout_rate'])
    finetune(model_generator, input_encoder, output_spec, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'], \
            seq_len = settings['seq_len'], batch_size = settings['batch_size'], max_epochs_per_stage = settings['max_epochs_per_stage'], \
            lr = settings['initial_lr_with_all_layers'], begin_with_frozen_pretrained_layers = True, lr_with_frozen_pretrained_layers = \
            settings['initial_lr_with_frozen_pretrained_layers'], n_final_epochs = 1, final_seq_len = settings['final_epoch_seq_len'], \
            final_lr = settings['final_epoch_lr'], callbacks = settings['training_callbacks'])

    for dataset_name, dataset in [('Training-set', train_set), ('Validation-set', valid_set), ('Test-set', test_set)]:

        log('*** %s performance: ***' % dataset_name)
        results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, output_spec, dataset['seq'], dataset['label'], \
                start_seq_len = settings['seq_len'], start_batch_size = settings['batch_size'])

        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(results)

        if confusion_matrix is not None:
            with pd.option_context('display.max_rows', 16, 'display.max_columns', 10):
                log('Confusion matrix:')
                display(confusion_matrix)

    return model_generator

In [ ]:
pretrained_model_generator, input_encoder = load_pretrained_model()


In [ ]:
run_benchmark('scop', pretrained_model_generator, input_encoder, pretraining_model_manipulation_function=get_model_with_hidden_layers_as_outputs)


[2024_07_19-11:48:47] ========== scop ==========
[2024_07_19-11:48:47] Output type: global categorical
[2024_07_19-11:48:47] 14112 training set records, 1568 validation set records, 3921 test set records.
[2024_07_19-11:48:47] 7 unique lebels.
[2024_07_19-11:48:47] Training set: Filtered out 6 of 14112 (0.0%) records of lengths exceeding 510.
[2024_07_19-11:48:48] Validation set: Filtered out 1 of 1568 (0.1%) records of lengths exceeding 510.
[2024_07_19-11:48:48] Training with frozen pretrained layers...
Epoch 1/40
441/441 [==============================] - ETA: 0s - loss: 0.5401

441/441 [==============================] - 54s 81ms/step - loss: 0.5401 - val_loss: 0.3108 - lr: 0.0100
Epoch 2/40
441/441 [==============================] - ETA: 0s - loss: 0.4464

441/441 [==============================] - 34s 76ms/step - loss: 0.4464 - val_loss: 0.3058 - lr: 0.0100
Epoch 3/40
441/441 [==============================] - ETA: 0s - loss: 0.4303

441/441 [==============================] - 34s 77ms/step - loss: 0.4303 - val_loss: 0.3002 - lr: 0.0100
Epoch 4/40
441/441 [==============================] - ETA: 0s - loss: 0.4226


Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
441/441 [==============================] - 34s 77ms/step - loss: 0.4226 - val_loss: 0.3294 - lr: 0.0100
Epoch 5/40
441/441 [==============================] - ETA: 0s - loss: 0.3256

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='d', y='e', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['a'].plot(kind='line', figsize=(8, 4), title='a')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['b'].plot(kind='line', figsize=(8, 4), title='b')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['c'].plot(kind='line', figsize=(8, 4), title='c')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['d'].plot(kind='line', figsize=(8, 4), title='d')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
log('Done.')


[2024_07_19-12:10:30] Done.


In [ ]:
import pandas as pd

train_counts = train_set['label'].value_counts()
valid_counts = valid_set['label'].value_counts()
test_counts = test_set['label'].value_counts()

print("Train Set Value Counts:\n", train_counts)
print("\nValidation Set Value Counts:\n", valid_counts)
print("\nTest Set Value Counts:\n", test_counts)


Train Set Value Counts:
 label
c    4197
d    3361
b    2961
a    2354
g     782
e     261
f     196
Name: count, dtype: int64

Validation Set Value Counts:
 label
c    466
d    373
b    329
a    262
g     87
e     29
f     22
Name: count, dtype: int64

Test Set Value Counts:
 label
c    1166
d     933
b     823
a     654
g     217
e      73
f      55
Name: count, dtype: int64


ProteinBERT provides a substantial boost in classification accuracy, achieving 0.88 compared to the 0.29 accuracy obtained with ProtT5. This improvement underscores ProteinBERT's effectiveness and suitability for protein family classification, demonstrating its advantage in capturing the complex patterns in protein sequences.